In [2]:
!pip install hopsworks==4.2.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.9/664.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import hopsworks
from google.colab import userdata

HOPSWORKS_API_KEY = userdata.get('HOPSWORKS_API_KEY')

print(f'API key loaded')


API key loaded


In [2]:
# ✅ Connect to your Hopsworks project
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()
print("✅ Connected to Hopsworks project successfully!")



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1251499
✅ Connected to Hopsworks project successfully!


In [3]:
import pandas as pd

def push_to_hopsworks(
    df: pd.DataFrame,
    fg_name: str,
    version: int,
    primary_key: list,
    event_time: str,
    description: str,
    online_enabled: bool = False
):
    """
    Create or update a Hopsworks Feature Group and insert data.
    """

    # Ensure datetime formatting
    df[event_time] = pd.to_datetime(df[event_time], errors="coerce", utc=True)
    df = df.dropna(subset=[event_time])
    df = df.reset_index(drop=True)

    # Create or get feature group
    fg = fs.get_or_create_feature_group(
        name=fg_name,
        version=version,
        primary_key=primary_key,
        event_time=event_time,
        description=description,
        online_enabled=online_enabled
    )

    # Insert data
    fg.insert(df, write_options={"wait_for_job": True})
    print(f"✅ Successfully pushed {len(df)} records to feature group: '{fg_name}' (v{version})")


# Loaded model_features_v1.csv to upload on Hopsworks (Historical data)

In [14]:
from google.colab import files

uploaded = files.upload()
print("Data loaded successfully!")

Saving model_features_v1.csv to model_features_v1.csv
Data loaded successfully!


In [4]:
historical_path = "model_features_v1.csv"

df_features = pd.read_csv(historical_path)
print(f"✅ Loaded {len(df_features)} historical records from {historical_path}")
print("Columns:", len(df_features.columns))

✅ Loaded 6577 historical records from model_features_v1.csv
Columns: 155


In [12]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 18.1 MB/s eta 0:00:00


In [5]:
push_to_hopsworks(
    df=df_features,
    fg_name="model_features",
    version=1,
    primary_key=["datetime"],
    event_time="datetime",
    description="Engineered AQI + Weather features (Karachi, hourly)",
    online_enabled=False
)

Uploading Dataframe: 100.00% |██████████| Rows 6577/6577 | Elapsed Time: 00:14 | Remaining Time: 00:00


Launching job: model_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1251499/jobs/named/model_features_1_offline_fg_materialization/executions
✅ Successfully pushed 6577 records to feature group: 'model_features' (v1)


# Loaded realtime_aqi_weather.csv to upload on Hopsworks (Real-time data)

In [6]:
from google.colab import files

uploaded = files.upload()
print("Data loaded successfully!")

Saving realtime_aqi_weather.csv to realtime_aqi_weather.csv
Data loaded successfully!


In [4]:
realtime_path = "realtime_aqi_weather.csv"

df_features = pd.read_csv(realtime_path)
print(f"✅ Loaded {len(df_features)} real-time records from {realtime_path}")
print("Columns:", len(df_features.columns))

✅ Loaded 1 real-time records from realtime_aqi_weather.csv
Columns: 12


In [5]:
df_features.rename(columns={"pm25": "pm2_5"}, inplace=True)

In [6]:
df_features["datetime"] = pd.to_datetime(df_features["datetime"], utc=True)
df_features["datetime_str"] = df_features["datetime"].astype(str)  # string key

In [7]:
push_to_hopsworks(
    df=df_features,
    fg_name="raw_observations",
    version=2,
    primary_key=["datetime_str"],
    event_time="datetime",
    description="Real-time AQI + Weather features (Karachi, hourly)",
    online_enabled=True,
)


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: raw_observations_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1251499/jobs/named/raw_observations_2_offline_fg_materialization/executions
✅ Successfully pushed 1 records to feature group: 'raw_observations' (v2)


# Deleting the current feature group

In [3]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

# Delete a specific feature group version
fg = fs.get_feature_group(name="aqi_features", version=1)
fg.delete()

print("✅ Deleted feature group 'aqi_features'")


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1251499


✅ Deleted feature group 'aqi_features'
